### Analyze the Crunchbase data set while keeping a memory profile of below 10MB of memory


In [21]:
import pandas as pd
import numpy as np

c_iter = pd.read_csv("crunchbase-investments.csv", chunksize = 5000, encoding='ISO-8859-1')

### Read and print first 5 rows:

In [71]:
df = pd.read_csv("crunchbase-investments.csv", nrows=5, encoding='ISO-8859-1')
df

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


### Find each column's missing value count:

In [33]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize = 5000, encoding="ISO-8859-1")
null_list = []

for c in c_iter:
    null_list.append(c.isnull().sum())

joined_list = pd.concat(null_list)
grouped_list = joined_list.groupby(joined_list.index).sum()
grouped_list.sort_values()


company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

### Find each column's memory footprint:

In [37]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize = 5000, encoding="ISO-8859-1")
mf_list = []

for c in c_iter:
    mf_list.append(c.memory_usage(deep=True) / 1024 ** 2 )
joined_list = pd.concat(mf_list)
grouped_list = joined_list.groupby(joined_list.index).sum()
grouped_list.sort_values()

Index                     0.000877
funded_year               0.403366
raised_amount_usd         0.403366
investor_category_code    0.593590
investor_state_code       2.361876
investor_country_code     2.524654
investor_city             2.751430
company_state_code        2.962161
company_country_code      3.025223
funded_quarter            3.226837
funded_month              3.226837
investor_region           3.238946
funding_round_type        3.252704
company_region            3.253541
company_category_code     3.262619
company_city              3.343512
funded_at                 3.378091
company_name              3.424955
investor_name             3.734270
company_permalink         3.869808
investor_permalink        4.749821
dtype: float64

### Find total memory footprint of all chunks combined:

In [26]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding="ISO-8859-1")
total_memory = 0
for c in c_iter:
    total_memory += c.memory_usage(deep=True).sum() / 1024 ** 2
total_memory

56.988484382629395

### Find total rows in data set:

In [38]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding="ISO-8859-1")
rowcount = 0
for c in c_iter:
    rowcount += c.size
rowcount

1057400

### Which columns can be dropped:
#### The investor_permalink and company_permalink columns aren't really necessary for analysis and they take up a relatively large amount of memory. I would drop those 2 columns to start. Next would be the investor_category_code as it is missing a large amount of data.


In [42]:
drop_cols = ["investor_permalink", "company_permalink", "investor_category_code"]
keep_cols = c.columns.drop(drop_cols)
keep_cols.tolist
print(keep_cols)
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding="ISO-8859-1", usecols=keep_cols)

Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')


### Identify the types of each column:

In [51]:
c_nums = c.select_dtypes(include=[np.number])
c_nums.info()
c_objs = c.select_dtypes(include=["object"])
c_objs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 50000 to 52869
Data columns (total 5 columns):
investor_country_code    0 non-null float64
investor_state_code      0 non-null float64
investor_city            0 non-null float64
funded_year              2870 non-null int64
raised_amount_usd        2580 non-null float64
dtypes: float64(4), int64(1)
memory usage: 112.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 50000 to 52869
Data columns (total 12 columns):
company_name             2870 non-null object
company_category_code    2860 non-null object
company_country_code     2870 non-null object
company_state_code       2841 non-null object
company_region           2870 non-null object
company_city             2825 non-null object
investor_name            2870 non-null object
investor_region          2870 non-null object
funding_round_type       2870 non-null object
funded_at                2870 non-null object
funded_month             2870 non-null ob

In [72]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding="ISO-8859-1", usecols=keep_cols)
vc_overall = {}
for c in c_iter:
    c_nums = c.select_dtypes(include=[np.number])
    columns = c_nums.columns
    for col in columns:
        if col in vc_overall:
            vc_overall[col] += c[col].value_counts()
        else:
            vc_overall[col] = c[col].value_counts()
    
vc_combined = pd.concat(vc_overall)
vc_grouped = vc_combined.groupby(vc_combined.index).sum()
vc_grouped


(funded_year, 1995.0)                   0.0
(funded_year, 1996.0)                   0.0
(funded_year, 1998.0)                   0.0
(funded_year, 1999.0)                  95.0
(funded_year, 2000.0)                 176.0
(funded_year, 2001.0)                 116.0
(funded_year, 2002.0)                 145.0
(funded_year, 2003.0)                   0.0
(funded_year, 2004.0)                 411.0
(funded_year, 2005.0)                2990.0
(funded_year, 2006.0)                3867.0
(funded_year, 2007.0)                4781.0
(funded_year, 2008.0)                4937.0
(funded_year, 2009.0)                4710.0
(funded_year, 2010.0)                6295.0
(funded_year, 2011.0)                8259.0
(funded_year, 2012.0)                8773.0
(funded_year, 2013.0)                7115.0
(raised_amount_usd, 2000.0)             0.0
(raised_amount_usd, 5000.0)             0.0
(raised_amount_usd, 6000.0)             0.0
(raised_amount_usd, 6500.0)             0.0
(raised_amount_usd, 10000.0)    

In [78]:
c_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000, encoding="ISO-8859-1", usecols=keep_cols)
vc_overall = {}
for c in c_iter:
    c_nums = c.select_dtypes(include=["object"])
    columns = c_nums.columns
    for col in columns:
        if col in vc_overall:
            vc_overall[col] += c[col].value_counts()
        else:
            vc_overall[col] = c[col].value_counts()
    
vc_combined = pd.concat(vc_overall)
vc_grouped = vc_combined.groupby(vc_combined.index).sum()
vc_grouped

(company_category_code, advertising)        3200.0
(company_category_code, analytics)          1863.0
(company_category_code, automotive)          164.0
(company_category_code, biotech)            4951.0
(company_category_code, cleantech)          1948.0
(company_category_code, consulting)          233.0
(company_category_code, design)                0.0
(company_category_code, ecommerce)          2168.0
(company_category_code, education)           783.0
(company_category_code, enterprise)         4489.0
(company_category_code, fashion)             368.0
(company_category_code, finance)             931.0
(company_category_code, games_video)        1893.0
(company_category_code, government)            0.0
(company_category_code, hardware)           1537.0
(company_category_code, health)              670.0
(company_category_code, hospitality)         331.0
(company_category_code, legal)                87.0
(company_category_code, local)                 0.0
(company_category_code, manufac

### Connect to SQLite database and load rows

In [79]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
c_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for c in c_iter:
    c.to_sql("investments", conn, if_exists='append', index=False)